#Загрузка и очистка текста

In [1]:
from google.colab import files
import re

# Загрузка файлов пользователем
uploaded = files.upload()  # Выберите все 4 файла: tom_1.txt, tom_2.txt, tom_3.txt, tom_4.txt

# Объединение текстов из всех файлов
full_text = ''
for filename in uploaded.keys():
    with open(filename, encoding='utf-8') as f:
        full_text += f.read()

# Приведение к нижнему регистру
full_text = full_text.lower()

# Оставляем только русские буквы и пробелы
cleaned_text = re.sub(r'[^а-яё ]', '', full_text)

# Удаляем лишние пробелы (например, множественные пробелы)
cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

# Выводим первые 500 символов для визуальной проверки
print(cleaned_text[:500])


Saving tom_1.txt to tom_1.txt
Saving tom_2.txt to tom_2.txt
Saving tom_3.txt to tom_3.txt
Saving tom_4.txt to tom_4.txt
лев толстойвойна и миртом том часть перваячасть ораячасть третья том лев николаевич толстойвойна и миртом часть первая е поместья мой верный раб ну что князь генуа и лукка стали не больше как поместьями фамилии бонапарте нет я вас предупреждаю если вы мне не скажете что у нас война если вы еще позволите себе защищать все гадости все ужасы этого антихриста право я верю что он антихрист я вас больше не знаю вы уж не друг мой вы уж не мой верный раб как вы говорите ну здравствуйте здравствуйте я ви


Построим матрицу переходных вероятностей — посимвольно (одна буква → следующая)

In [2]:
from collections import defaultdict

# Шаг 1: считаем частоты переходов символов
transition_counts = defaultdict(lambda: defaultdict(int))

# Проходим по всем парам символов
for i in range(len(cleaned_text) - 1):
    current_char = cleaned_text[i]
    next_char = cleaned_text[i + 1]
    transition_counts[current_char][next_char] += 1


In [3]:
# Преобразуем частоты в вероятности
transition_probs = {}

for current_char, next_chars in transition_counts.items():
    total = sum(next_chars.values())
    transition_probs[current_char] = {
        char: count / total for char, count in next_chars.items()
    }


Проверка (например, для буквы "в")

In [4]:
for next_char, prob in transition_probs['в'].items():
    print(f"'в' → '{next_char}': {prob:.4f}")


'в' → ' ': 0.1633
'в' → 'о': 0.1900
'в' → 'а': 0.1390
'в' → 'и': 0.0790
'в' → 'е': 0.1116
'в' → 'ы': 0.0638
'в' → 'с': 0.0785
'в' → 'у': 0.0206
'в' → 'л': 0.0264
'в' → 'н': 0.0284
'в' → 'ш': 0.0300
'в' → 'ь': 0.0041
'в' → 'р': 0.0161
'в' → 'я': 0.0052
'в' → 'б': 0.0004
'в' → 'д': 0.0078
'в' → 'м': 0.0034
'в' → 'з': 0.0125
'в' → 'к': 0.0035
'в' → 'п': 0.0063
'в' → 'т': 0.0051
'в' → 'х': 0.0011
'в' → 'в': 0.0010
'в' → 'г': 0.0010
'в' → 'ц': 0.0003
'в' → 'ч': 0.0010
'в' → 'ъ': 0.0003
'в' → 'э': 0.0000
'в' → 'щ': 0.0001
'в' → 'ю': 0.0000
'в' → 'ж': 0.0000
'в' → 'ё': 0.0000


#Найти вероятность появления строки из 4 букв, начинающейся на «П» и заканчивающей на «Р»

In [5]:
total_probability = 0

# Все возможные символы в алфавите (те, что реально встречаются)
alphabet = list(transition_probs.keys())

for c1 in alphabet:
    for c2 in alphabet:
        try:
            # Вероятность перехода п → c1 → c2 → р
            prob = (transition_probs['п'][c1] *
                    transition_probs[c1][c2] *
                    transition_probs[c2]['р'])
            total_probability += prob
        except KeyError:
            # Если какой-то переход невозможен — пропускаем
            continue

print(f"Суммарная вероятность строки вида 'п _ _ р': {total_probability:.8f}")


Суммарная вероятность строки вида 'п _ _ р': 0.03253130


Вероятность встретить строку длины 4, начинающуюся на «п» и заканчивающуюся на «р», по построенной матрице переходных вероятностей составляет:

0.0325 ≈ 3.25%

#Сгенерируем слово длиной 8 символов, начинающееся с «б», используя нашу матрицу как цепь Маркова.

In [6]:
import random

# Начальная буква
current_char = 'б'
generated = current_char

# Генерируем ещё 7 букв по переходной вероятности
for _ in range(7):
    next_chars = list(transition_probs[current_char].keys())
    probabilities = list(transition_probs[current_char].values())
    current_char = random.choices(next_chars, weights=probabilities)[0]
    generated += current_char

print(f"Сгенерированное слово: {generated}")


Сгенерированное слово: бей сиве


Чтобы получить реально "одно слово" без пробелов, мы просто будем игнорировать все переходы, ведущие к пробелу во время генерации.

In [7]:
import random

# Начинаем с буквы 'б'
current_char = 'б'
generated = current_char

for _ in range(7):
    # Получаем возможные следующие символы и вероятности
    next_chars = []
    probs = []

    for char, prob in transition_probs[current_char].items():
        if char != ' ':  # игнорируем пробелы
            next_chars.append(char)
            probs.append(prob)

    # Если нет куда идти — прерываем
    if not next_chars:
        break

    # Выбираем следующий символ
    current_char = random.choices(next_chars, weights=probs)[0]
    generated += current_char

print(f"Слово без пробелов: {generated}")


Слово без пробелов: брышлыве
